# Profiling

**[Original tutorial](https://docs.pymc.io/notebooks/profiling.html)**

In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotnine as gg
import pymc3 as pm

%config InlineBackend.figure_format = "retina"
az.style.use("arviz-darkgrid")
gg.theme_set(gg.theme_minimal)
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)

PyMC3 has wrapped profiling tools from Theano into `model.profile()`.
This function returns a `ProfileStats` object with information about the underlying Theano operations.

Below is an example of profiling the likelihood and gradient for the stochastic volatility example.

In [2]:
returns = pd.read_csv(pm.get_data("SP500.csv"), index_col=0, parse_dates=True)
returns.head()

,Close,change
Date,,
2008-05-02,1413.900024,0.003230
2008-05-05,1407.489990,-0.004544
2008-05-06,1418.260010,0.007623
2008-05-07,1392.569946,-0.018280
2008-05-08,1397.680054,0.003663


In [3]:
with pm.Model() as model:
    sigma = pm.Exponential("sigma", 1.0 / 0.02, testval=0.1)
    nu = pm.Exponential("nu", 1.0 / 10.0)
    s = pm.GaussianRandomWalk("s", sigma ** -2, shape=returns.shape[0])
    r = pm.StudentT("r", nu, lam=np.exp(-2 * s), observed=returns["change"])

To profile, call the `profile()` function and summarise the return values.

In [4]:
model.profile(model.logpt).summary()

Function profiling
  Message: /usr/local/Caskroom/miniconda/base/envs/pymc3-tutorials/lib/python3.9/site-packages/pymc3/model.py:1191
  Time in 1000 calls to Function.__call__: 1.321383e-01s
  Time in Function.fn.__call__: 1.078756e-01s (81.638%)
  Time in thunks: 9.906149e-02s (74.968%)
  Total compile time: 1.016328e+00s
    Number of Apply nodes: 22
    Theano Optimizer time: 8.974490e-01s
       Theano validate time: 2.188683e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 5.971813e-02s
       Import time 3.460383e-02s
       Node make_thunk time 5.846930e-02s
           Node Elemwise{Composite{log((i0 * i1))}}(TensorConstant{(1,) of 0...4309189535}, Elemwise{Composite{inv(sqr(i0))}}[(0, 0)].0) time 7.901907e-03s
           Node Elemwise{Composite{Switch(Cast{int8}((GT(Composite{exp((i0 * i1))}(i0, i1), i2) * i3 * GT(inv(sqrt(Composite{exp((i0 * i1))}(i0, i1))), i2))), (((i4 + (i5 * log(((i6 * Composite{exp((i0 * i1))}(i0, i1)) / i7)))) - i8) - (i5 * i9 * 

We can also profile the gradient call `dlogp / dx`.

In [5]:
model.profile(pm.gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /usr/local/Caskroom/miniconda/base/envs/pymc3-tutorials/lib/python3.9/site-packages/pymc3/model.py:1191
  Time in 1000 calls to Function.__call__: 1.813309e-01s
  Time in Function.fn.__call__: 1.566050e-01s (86.364%)
  Time in thunks: 1.386719e-01s (76.474%)
  Total compile time: 2.309332e+00s
    Number of Apply nodes: 47
    Theano Optimizer time: 1.987331e+00s
       Theano validate time: 6.048441e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 2.872701e-01s
       Import time 6.456828e-02s
       Node make_thunk time 2.851560e-01s
           Node Elemwise{Composite{Switch(i0, (i1 * (i2 - i3)), i4)}}(Elemwise{Composite{Cast{int8}(GT(i0, i1))}}.0, InplaceDimShuffle{x}.0, Subtensor{int64::}.0, Subtensor{:int64:}.0, TensorConstant{(1,) of 0}) time 1.713119e-01s
           Node Elemwise{Composite{Switch(i0, (i1 * (i2 + ((i3 * i4 * i5 * i6) / i7))), i8)}}[(0, 6)](Elemwise{Composite{Cast{int8}((GT(i0, i1) * i2 * GT(inv(sqrt(i0)), i1)

---

In [6]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Tue Feb 02 2021

Python implementation: CPython
Python version       : 3.9.1
IPython version      : 7.20.0

pymc3     : 3.9.3
plotnine  : 0.7.1
arviz     : 0.11.0
numpy     : 1.20.0
matplotlib: 3.3.4
pandas    : 1.2.1

Watermark: 2.1.0

